# EXTRACCION, TRANSFORMACIÓN Y LIMPIEZA 

## Base de datos: Movies

Se utiliza la libreria de pandas para dar inicio a la estracción y tratamiento de los datos.

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import os

Inicio la extracción, primero del detaset "movies" y luego del dataset "credits" para despues unirlos. 

In [2]:
# Extraigo el dataset "movies" y reviso el tamaño
df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
df_movies.shape

C:\Users\anavi\AppData\Local\Temp\ipykernel_14348\131553651.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')


(45466, 24)

In [3]:
# Extraigo los dataset "credits" y reviso el tamaño
df_credits = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')
df_credits.shape

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [4]:
# Quiero saber el tamaño de mi dataset para reducirlo hasta llegar a 100 megas
tamaño1 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
tamaño2 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')

print(f"Tamaño movies: {tamaño1 / (1024 * 1024):.2f} MB y tamaño credits: {tamaño2 / (1024 * 1024):.2f} MB")

Tamaño movies: 32.85 MB y tamaño credits: 181.12 MB


Como veo que el dataset credits es mucho más grande que el dataset movies entonces empiezo limpiandolo para reducirlo al máximo.

In [5]:
df_credits[df_credits.duplicated(keep=False)].shape # Verifico si hay datos duplicados para eliminarlos posteriormente

(73, 3)

In [6]:
tamaño_en_bytes = df_credits.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 256.51 MB.


In [7]:
df_credits = df_credits.drop_duplicates() # Elimino los datos duplicados en mi dataset original
df_credits[df_credits.duplicated(keep=False)].shape #Verifico que hayan quedado eliminados

(0, 3)

In [8]:
df_credits.columns # Verifico que columnas existen en mi data set

Index(['cast', 'crew', 'id'], dtype='object')

In [9]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Mi dataset tiene dos columas. La primera contiene el elenco "cast" y la segunda contiene los nombres del equipo "crew". Voy a empezar por depurar la columna "cast" y dejar unicamente 5 actores principales, respetando el orden en el que vienen.

In [10]:
type(df_credits['cast'][0]) # se verifica el tipo de datos, deben ser listas o dicccionarios para poder normalizarlos

str

In [11]:
#Se convierte el tipo de datos de String a Diccionario o lista

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_credits['cast'] = df_credits['cast'].apply(convertir_a_diccionario)

type(df_credits['cast'][0])

list

In [12]:
# Se muestran los tipos de datos únicos en la columnas

df_credits['cast'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [13]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_credits.iterrows():
    entry = row['cast']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_cast = pd.concat(expanded_dfs, ignore_index=True)

In [14]:
df_cast.columns #Para verificar las columnas existentes en el nuevo data frame

Index(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order',
       'profile_path'],
      dtype='object')

In [15]:
df_cast.shape #Para verificar el tamaño del nuevo data frame

(562132, 8)

In [16]:
df_cast = df_cast[df_cast['order'] == 1] # Filtrar el actor principal,  o el que sale de primer en el casty eliminar el resto

In [17]:
df_cast.tail(10)

,cast_id,character,credit_id,gender,id,name,order,profile_path
562041,5.0,,52fe475bc3a368484e0c2a91,2.0,67179,Giulio Brogi,1,/z6SzkWIDsQ5zo35kFD6PmcKmxt4.jpg
562045,5.0,The Creeper,52fe48f49251416c9109dceb,2.0,84419,Rondo Hatton,1,/zir0SuHwhFb2Sj7feQgIeMecG8m.jpg
562056,30.0,Debuty Hank Hart,56ff777092514167870015f6,0.0,390959,Tony Abatemarco,1,/pGobBL7YIG2sQDTEZZnOHTfgTSD.jpg
562075,2.0,Branwall,5403d707c3a368068c005b39,2.0,289923,Monty Bane,1,/7myDNmmd1GYHeMRArJH6vsQJ7Yk.jpg
562083,1.0,Kira (as Cassandra Leigh),52fe4dfe9251416c75147e0b,1.0,222848,Lisa Boyle,1,/r7YqIRBbeNIgVNK6xgmrmLDh4uW.jpg
562093,1.0,Sir Robert Hode,52fe44439251416c9100a887,2.0,30840,Patrick Bergin,1,/mxFum2Fr5YWD6kDQYb5u7EEvuva.jpg
562098,0.0,,5894a909925141427e0079a5,1.0,439050,Leila Hatami,1,/ppI1Q4a7zAXTaAyPRSvPm0vhove.jpg
562101,1002.0,Sister Angela,52fe4af1c3a36847f81e9b1f,1.0,111109,Angel Aquino,1,/aZLCxPbokn0EX1cNXxF9UCaA8gP.jpg
562112,6.0,Emily Shaw,52fe4776c3a368484e0c83a3,1.0,67758,Erika Eleniak,1,/7ZdUGY78qpB2MkTzAQvYEYzncwN.jpg
562127,2.0,,52fe4ea59251416c7515d7d5,2.0,227506,Iwan Mosschuchin,1,None


In [18]:
df_cast = df_cast[['id'] + [col for col in df_cast.columns if col != 'id']] #Dejo de primera la columna "id"
df_cast = df_cast.drop(columns=['credit_id', 'gender', 'cast_id', 'profile_path', 'order']) #Elimino las columnas innecesarias

In [19]:
df_cast.rename(columns={'id': 'Id', 'name': 'Lead actor', 'character': 'Character Name'}, inplace=True)


In [20]:
df_cast.shape #Para verificar el tamaño nuevamente

(43025, 3)

In [21]:
df_cast.head(10)

,Id,Character Name,Lead actor
0,862,Woody (voice),Tom Hanks
13,8844,Alan Parrish,Robin Williams
39,15602,Max Goldman,Walter Matthau
46,31357,Savannah 'Vannah' Jackson,Whitney Houston
56,11862,George Banks,Steve Martin
68,949,Lt. Vincent Hanna,Al Pacino
133,11860,Linus Larrabee,Harrison Ford
190,45325,Tom Sawyer,Jonathan Taylor Thomas
197,9091,Darren Francis Thomas McCord,Jean-Claude Van Damme
203,710,James Bond,Pierce Brosnan


In [22]:
df_cast.shape

(43025, 3)

In [23]:
# Calcular el uso de memoria del DataFrame en bytes
uso_memoria = df_cast.memory_usage(deep=True).sum()

# Convertir el uso de memoria a megabytes
uso_memoria_mb = uso_memoria / (1024 * 1024)

print(f"El DataFrame utiliza aproximadamente {uso_memoria_mb:.2f} MB de memoria.")


El DataFrame utiliza aproximadamente 6.45 MB de memoria.


In [24]:
tamaño_en_bytes = df_cast.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 6.45 MB.


Se crea un nuevo dataset con la información con los cinco actores y actrices principales, quedaria pendiente de ver si hay que agregar a  este dataset  algun dato especifico para las funciones.

In [25]:
df_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [26]:
df_cast.to_csv('cast.csv', index=False)

Una vez eliminadas las celdas que no se van a usar de la columna cast y creado un data set con este nombre, se procede a trabajar con la columna "Crew" de la cual solo se va a conservar el Director. 

In [27]:
df_credits['crew'].head()

0    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
1    [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...
2    [{'credit_id': '52fe466a9251416c75077a89', 'de...
3    [{'credit_id': '52fe44779251416c91011acb', 'de...
4    [{'credit_id': '52fe44959251416c75039ed7', 'de...
Name: crew, dtype: object

In [28]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_credits['crew'] = df_credits ['crew'].apply(convertir_a_diccionario)

type(df_credits['crew'][0])

list

In [29]:
# Se muestran los tipos de datos únicos en la columna

df_credits['crew'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [30]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_credits.iterrows():
    entry = row['crew']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_director = pd.concat(expanded_dfs, ignore_index=True)


In [31]:
df_director.head() #Para ver como quedó estructurado el dataset creado. 

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,862,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,862,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,862,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,862,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [32]:
df_director.shape #Para confirmar el tamaño (filas y columnas)

(463927, 7)

In [33]:
# Para revisar el tamaño en megas. 

tamaño_en_bytes = df_director.memory_usage(deep=True).sum()

# Convertir a megabytes
tamaño_en_megabytes = tamaño_en_bytes / (1024 ** 2)

print(f"El tamaño del dataset es de {tamaño_en_megabytes:.2f} MB.")

El tamaño del dataset es de 150.73 MB.


In [34]:
df_director = df_director[df_director['job'] == 'Director']

In [35]:
df_director.columns

Index(['credit_id', 'department', 'gender', 'id', 'job', 'name',
       'profile_path'],
      dtype='object')

In [36]:
df_director.head()

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
109,52fe44bfc3a36847f80a7c7d,Directing,2.0,8844,Director,Joe Johnston,/fok4jaO62v5IP6hkpaaAcXuw2H.jpg
122,52fe466a9251416c75077a89,Directing,2.0,15602,Director,Howard Deutch,/68Vae1HkU1NxQZ6KEmuxIpno7c9.jpg
126,52fe44779251416c91011acb,Directing,2.0,31357,Director,Forest Whitaker,/4pMQkelS5lK661m9Kz3oIxLYiyS.jpg
141,52fe44959251416c75039eef,Directing,2.0,11862,Director,Charles Shyer,/hnWGd74CbmTcDCFQiJ8SYLazIXW.jpg


In [37]:
df_director = df_director.drop(columns=['credit_id', 'profile_path', 'department', 'gender', 'job'])
df_director.rename(columns={'id': 'Id', 'name': 'Director'}, inplace=True)

In [38]:
df_director.head()

,Id,Director
0,862,John Lasseter
109,8844,Joe Johnston
122,15602,Howard Deutch
126,31357,Forest Whitaker
141,11862,Charles Shyer


In [39]:
df_director.shape

(49008, 2)

In [40]:
df_crew = pd.merge(df_cast, df_director, on='Id', how='outer')
df_crew = df_crew.drop_duplicates(subset='Id')

In [41]:
df_crew.head()

,Id,Character Name,Lead actor,Director
0,2,Taisto Olavi Kasurinen,Turo Pajala,Aki Kaurismäki
1,3,Nikander,Matti Pellonpää,Aki Kaurismäki
2,5,Ted the Bellhop,Tim Roth,Allison Anders
6,6,Frank Wyatt,Emilio Estevez,Stephen Hopkins
7,11,Luke Skywalker,Mark Hamill,George Lucas


In [42]:
df_crew.shape

(44995, 4)

In [43]:
df_crew[df_crew['Id'] == 862]


,Id,Character Name,Lead actor,Director
727,862,Woody (voice),Tom Hanks,John Lasseter


Este resultado voy a agregarlo al dataset de Movies,   pero primero voy a hacer  las transformaciones necesarias para limpiar este dataset y reducirlo lo más posible. 

In [44]:
df_movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [45]:
df_movies[df_movies.duplicated(keep=False)].shape

(25, 24)

In [46]:
df_movies = df_movies.drop_duplicates()

In [47]:
df_movies.shape

(45453, 24)

In [48]:
df_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [49]:
# Se eliminan las columnas para aligerar el tamaño del dataset: video, imbd_id, poster_path y homepage (solicitadas)

df_movies = df_movies.drop(columns = ['adult', 'imdb_id', 'poster_path', 'homepage', 'video', ], errors='ignore')

df_movies.columns # Para mostrar las columnas con las que se va a trabajar

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [50]:
# Se eliminan las columnas para aligerar el tamaño del dataset (no utilizadas)

df_movies = df_movies.drop(columns = ['homepage', 'original_language', 'overview','runtime', 'spoken_languages', 'status', 'tagline', 'production_countries'], errors='ignore')

df_movies.columns # Para mostrar las columnas con las que se va a trabajar

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_title',
       'popularity', 'production_companies', 'release_date', 'revenue',
       'title', 'vote_average', 'vote_count'],
      dtype='object')

In [51]:
df_movies.shape

(45453, 12)

Se empieza desde aqui a hacer la depuración de las columnas que se encuentran anidadas:
1. belongs_to_collection
2. genres
3. production_companies
4. production_countries

DEPURACIÓN COLUMNA 'belongs_to_collection'

In [52]:
# Se verifica si la columna tiene valores nulos

df_movies['belongs_to_collection'].isnull().sum()

np.int64(40959)

In [53]:
# La columna tiene datos nulos, se reemplazan con listas vacías

df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].fillna('[]')
df_movies['belongs_to_collection'].head(10)

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                   []
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                   []
4    {'id': 96871, 'name': 'Father of the Bride Col...
5                                                   []
6                                                   []
7                                                   []
8                                                   []
9    {'id': 645, 'name': 'James Bond Collection', '...
Name: belongs_to_collection, dtype: object

In [54]:
type(df_movies['belongs_to_collection'][0])

str

In [55]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(convertir_a_diccionario)

type(df_movies['belongs_to_collection'][0])

dict

In [56]:
# Se muestran los tipos de datos únicos en la columna

df_movies['belongs_to_collection'].apply(type).unique()

array([<class 'dict'>, <class 'list'>, <class 'float'>], dtype=object)

In [57]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_movies.iterrows():
    entry = row['belongs_to_collection']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_collection = pd.concat(expanded_dfs, ignore_index=True)

In [58]:
df_collection.columns

Index(['id', 'name', 'poster_path', 'backdrop_path'], dtype='object')

In [59]:
df_collection.shape

(4491, 4)

In [60]:
df_collection = df_collection.rename(columns={'name': 'Collection'}) #Cambio el nombre para dejarlo en el data frame final
df_collection = df_collection.drop_duplicates() #elimino los duplicados del df resultante
df_collection[df_collection['id'].duplicated(keep=False)] #Confirmo si quedan duplicados

,id,Collection,poster_path,backdrop_path


In [61]:
df_movies = df_movies.drop(columns=['belongs_to_collection']) #Elimino la columna que ya dupliqueé
df_movies = pd.merge(df_movies, df_collection[['id', 'Collection']], on='id', how='left') #Hago la combinación de datos

In [62]:
df_movies.shape

(45453, 12)

In [63]:
df_movies.columns

Index(['budget', 'genres', 'id', 'original_title', 'popularity',
       'production_companies', 'release_date', 'revenue', 'title',
       'vote_average', 'vote_count', 'Collection'],
      dtype='object')

In [64]:
df_movies.head(1)

,budget,genres,id,original_title,popularity,production_companies,release_date,revenue,title,vote_average,vote_count,Collection
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection


DEPURACIÓN COLUMNA 'genres'

In [65]:
# Se verifica si la columna tiene valores nulos
 
df_movies['genres'].isnull().sum()

np.int64(0)

In [66]:
type(df_movies['genres'][0])

str

In [67]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_movies['genres'] = df_movies['genres'].apply(convertir_a_diccionario)

type(df_movies['genres'][0])

list

In [68]:
# Se muestran los tipos de datos únicos en la columna
df_movies['genres'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [69]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_movies.iterrows():
    entry = row['genres']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_genres = pd.concat(expanded_dfs, ignore_index=True)

In [70]:
df_genres.shape

(91074, 3)

In [71]:
df_genres.head()

,id,name,order
0,862,Animation,1
1,862,Comedy,2
2,862,Family,3
3,8844,Adventure,1
4,8844,Fantasy,2


In [72]:
# Crear una función para combinar valores únicos en una lista
def combine_unique(series):
    return list(series.unique())

# Agrupar por 'id' y combinar valores únicos en listas para las columnas 'job' y otras relevantes
df_grouped = df_genres.groupby('id').agg({
    'name': combine_unique,
}).reset_index()

df_grouped.head()

,id,name
0,100,"[Comedy, Crime]"
1,10000,"[Comedy, Drama]"
2,10001,"[Comedy, Science Fiction]"
3,100010,"[Drama, War]"
4,100017,[Drama]


In [73]:
df_movies.columns

Index(['budget', 'genres', 'id', 'original_title', 'popularity',
       'production_companies', 'release_date', 'revenue', 'title',
       'vote_average', 'vote_count', 'Collection'],
      dtype='object')

In [74]:
df_movies = df_movies.drop(columns=['genres'])
df_grouped = df_grouped.rename(columns={'name': 'genres'})
df_movies = pd.merge(df_movies, df_grouped[['id', 'genres']], on='id', how='left')

In [75]:
df_movies.head(3)

,budget,id,original_title,popularity,production_companies,release_date,revenue,title,vote_average,vote_count,Collection,genres
0,30000000,862,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]",1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]"
1,65000000,8844,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]"
2,0,15602,Grumpier Old Men,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...",1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]"


In [76]:
df_movies.shape

(45453, 12)

In [77]:
df_movies.columns

Index(['budget', 'id', 'original_title', 'popularity', 'production_companies',
       'release_date', 'revenue', 'title', 'vote_average', 'vote_count',
       'Collection', 'genres'],
      dtype='object')

DEPURACIÓN COLUMNA 'production_companies'

In [78]:
# Se verifica si la columna tiene valores nulos
 
df_movies['production_companies'].isnull().sum()

np.int64(3)

In [79]:
# La columna tiene datos nulos, se reemplazan con listas vacías

df_movies['production_companies'] = df_movies['production_companies'].fillna('[]')
df_movies['production_companies'].head(10)

0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3    [{'name': 'Twentieth Century Fox Film Corporat...
4    [{'name': 'Sandollar Productions', 'id': 5842}...
5    [{'name': 'Regency Enterprises', 'id': 508}, {...
6    [{'name': 'Paramount Pictures', 'id': 4}, {'na...
7          [{'name': 'Walt Disney Pictures', 'id': 2}]
8    [{'name': 'Universal Pictures', 'id': 33}, {'n...
9    [{'name': 'United Artists', 'id': 60}, {'name'...
Name: production_companies, dtype: object

In [80]:
type(df_movies['production_companies'][0])

str

In [81]:
#Se convierte el tipo de datos de String a Diccionario

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_movies['production_companies'] = df_movies['production_companies'].apply(convertir_a_diccionario)

type(df_movies['production_companies'][0])

list

In [82]:
# Se muestran los tipos de datos únicos en la columna
df_movies['production_companies'].apply(type).unique()

array([<class 'list'>, <class 'bool'>], dtype=object)

In [83]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_movies.iterrows():
    entry = row['production_companies']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_prcompany = pd.concat(expanded_dfs, ignore_index=True)

In [84]:
df_movies.shape

(45453, 12)

In [85]:
df_prcompany['order'].value_counts().sort_index()

order
1     33575
2     17001
3      9039
4      4325
5      2378
6      1428
7       867
8       553
9       373
10      240
11      173
12      131
13       97
14       78
15       62
16       53
17       32
18       25
19       22
20       17
21       12
22        8
23        5
24        5
25        4
26        3
Name: count, dtype: int64

In [86]:
df_prcompany.rename(columns={'name': 'Production_Company'}, inplace=True) #Cambio el nombre "name" por "Product_Company"
df_prcompany = df_prcompany[~df_prcompany['order'].between(4, 26)] #Elimino las filas para las peliculas que tengan más de 4 productoras.

In [87]:
df_prcompany = df_prcompany.drop_duplicates(subset=['id', 'order']) # Para eliminar duplicados
df_prcompany = df_prcompany.pivot(index='id', columns='order', values='Production_Company') # Crea una tabla pivote  para separar las filas en col
df_prcompany.columns = ['ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'] #Para dejar claros los nuevos nombres
df_prcompany.reset_index(inplace=True) #Se resetea el indice. 

In [88]:
df_prcompany.head()

,id,ProdCompany_1,ProdCompany_2,ProdCompany_3
0,100,Handmade Films Ltd.,Summit Entertainment,PolyGram Filmed Entertainment
1,10000,Ministère de la Culture et de la Francophonie,Ministère des Affaires Étrangères,NaN
2,10001,Warner Bros.,NaN,NaN
3,100010,Metro-Goldwyn-Mayer (MGM),NaN,NaN
4,100017,MMM Film Zimmermann & Co,NaN,NaN


In [89]:
df_prcompany.columns

Index(['id', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'], dtype='object')

In [90]:
df_movies.columns

Index(['budget', 'id', 'original_title', 'popularity', 'production_companies',
       'release_date', 'revenue', 'title', 'vote_average', 'vote_count',
       'Collection', 'genres'],
      dtype='object')

In [91]:
df_movies = df_movies.drop(columns=['production_companies'])
df_movies = pd.merge(df_movies, df_prcompany[['id', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3']], on='id', how='left')

In [92]:
df_movies.columns

Index(['budget', 'id', 'original_title', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'Collection',
       'genres', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'],
      dtype='object')

In [93]:
df_movies.shape

(45453, 14)

Una vez terminada la depuración de columnas anidadas, se verifican las columnas resultantes y la estructura del nuevo dataset. Se opta por no borrar columnas en esta etapa del proyecto, posteriormente se evaluará si se van a eliminar. 

In [94]:
df_movies.columns # Para mostrar las columnas resultantes

Index(['budget', 'id', 'original_title', 'popularity', 'release_date',
       'revenue', 'title', 'vote_average', 'vote_count', 'Collection',
       'genres', 'ProdCompany_1', 'ProdCompany_2', 'ProdCompany_3'],
      dtype='object')

In [95]:
df_movies.head(3) # para mostrar la nueva estructura del dataset "movies_dataset.csv"

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3
0,30000000,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN
1,65000000,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications
2,0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN


Ahora voy a incluir en este dataset la información de la columna director y del actor principal tomando la información del dataset df_crew, a traves de un merge.

In [96]:
df_movies['id'] = df_movies['id'].astype(str)
df_crew['Id'] = df_crew['Id'].astype(str)

In [97]:
df_movies = pd.merge(df_movies, df_crew, how='left', left_on='id', right_on='Id') # se realiza el merge
df_movies = df_movies.drop(columns=['Id']) # Me quedó creada la colmna Id, la elimino.


In [98]:
df_movies.head()

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3,Character Name,Lead actor,Director
0,30000000,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN,Woody (voice),Tom Hanks,John Lasseter
1,65000000,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications,Alan Parrish,Robin Williams,Joe Johnston
2,0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN,Max Goldman,Walter Matthau,Howard Deutch
3,16000000,31357,Waiting to Exhale,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,NaN,"[Comedy, Drama, Romance]",Twentieth Century Fox Film Corporation,NaN,NaN,Savannah 'Vannah' Jackson,Whitney Houston,Forest Whitaker
4,0,11862,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,[Comedy],Sandollar Productions,Touchstone Pictures,NaN,George Banks,Steve Martin,Charles Shyer


DESPUES DE ESTO, SE INICIAL LAS TRANSFORMACIONES SUGERIDAS PARA EL PROYECTO. 

In [99]:
# Se reemplazan con cero los valores nulos en las columnas "revenue" y "budget" 
df_movies['revenue'] = df_movies['revenue'].fillna(0)
df_movies['budget'] = df_movies['budget'].fillna(0)

In [100]:
#Se eliminan los valores nulos de la columna "release_date"
df_movies = df_movies.dropna(subset=['release_date'])

In [101]:
#Se verifican como quedaron los cambios en estas dos columnas
df_movies[['revenue', 'budget', 'release_date']].tail(20)

,revenue,budget,release_date
45432,0.0,0,1906-01-01
45433,0.0,0,1909-01-01
45434,0.0,0,1904-03-05
45435,0.0,0,1904-01-01
45436,0.0,0,2005-09-13
45437,0.0,0,1900-01-01
45438,0.0,0,1900-01-01
45439,0.0,0,1981-01-01
45440,0.0,0,2017-07-07
45441,0.0,0,2015-01-11


In [102]:
# Se convierte la columna 'release_date' al formato de fecha AAAA-mm-dd
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce')
df_movies['release_date'].head()

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]

In [103]:
# Se crea la columna 'release_year' extrayendo el año de 'release_date' como cadena de texto
df_movies['release_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)
df_movies['release_year'] = df_movies['release_date'].dt.year.astype(int).astype(str)
df_movies['release_year'].head()

C:\Users\anavi\AppData\Local\Temp\ipykernel_10736\3005991044.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_movies['release_date'].fillna(pd.Timestamp('1900-01-01'), inplace=True)


0    1995
1    1995
2    1995
3    1995
4    1995
Name: release_year, dtype: object

In [104]:
# El codigo permite asegurarse de que las columnas 'revenue' y 'budget' sean numéricas
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce').fillna(0)
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce').fillna(0)

# Se crea la columna 'return' calculando el retorno de inversión
df_movies['return'] = df_movies.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] > 0 else 0, axis=1)

# Ultimo, se verifica las primeras filas de la nueva columna 'return'
print(df_movies[['revenue', 'budget', 'return']].head())

       revenue      budget     return
0  373554033.0  30000000.0  12.451801
1  262797249.0  65000000.0   4.043035
2          0.0         0.0   0.000000
3   81452156.0  16000000.0   5.090760
4   76578911.0         0.0   0.000000


In [105]:
df_movies.head(3)

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3,Character Name,Lead actor,Director,release_year,return
0,30000000.0,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN,Woody (voice),Tom Hanks,John Lasseter,1995,12.451801
1,65000000.0,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications,Alan Parrish,Robin Williams,Joe Johnston,1995,4.043035
2,0.0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN,Max Goldman,Walter Matthau,Howard Deutch,1995,0.000000


In [106]:
df_movies.shape

(45366, 19)

In [107]:
df_movies.head()

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3,Character Name,Lead actor,Director,release_year,return
0,30000000.0,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN,Woody (voice),Tom Hanks,John Lasseter,1995,12.451801
1,65000000.0,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications,Alan Parrish,Robin Williams,Joe Johnston,1995,4.043035
2,0.0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN,Max Goldman,Walter Matthau,Howard Deutch,1995,0.000000
3,16000000.0,31357,Waiting to Exhale,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,NaN,"[Comedy, Drama, Romance]",Twentieth Century Fox Film Corporation,NaN,NaN,Savannah 'Vannah' Jackson,Whitney Houston,Forest Whitaker,1995,5.090760
4,0.0,11862,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,[Comedy],Sandollar Productions,Touchstone Pictures,NaN,George Banks,Steve Martin,Charles Shyer,1995,0.000000


In [108]:
df_movies['release_year'] = pd.to_numeric(df_movies['release_year'], errors='coerce') #El año lo tengo como texto, lo paso a numero.

Una vez finalizada la limpiez, se extrae el dataset completamente depurado para que se vean los resultados. 

In [109]:
df_movies.head() # Se revisan los datos que quedaron, con los cuales se van a trabajar las funciones

,budget,id,original_title,popularity,release_date,revenue,title,vote_average,vote_count,Collection,genres,ProdCompany_1,ProdCompany_2,ProdCompany_3,Character Name,Lead actor,Director,release_year,return
0,30000000.0,862,Toy Story,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,Toy Story Collection,"[Animation, Comedy, Family]",Pixar Animation Studios,NaN,NaN,Woody (voice),Tom Hanks,John Lasseter,1995,12.451801
1,65000000.0,8844,Jumanji,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,NaN,"[Adventure, Fantasy, Family]",TriStar Pictures,Teitler Film,Interscope Communications,Alan Parrish,Robin Williams,Joe Johnston,1995,4.043035
2,0.0,15602,Grumpier Old Men,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,Grumpy Old Men Collection,"[Romance, Comedy]",Warner Bros.,Lancaster Gate,NaN,Max Goldman,Walter Matthau,Howard Deutch,1995,0.000000
3,16000000.0,31357,Waiting to Exhale,3.859495,1995-12-22,81452156.0,Waiting to Exhale,6.1,34.0,NaN,"[Comedy, Drama, Romance]",Twentieth Century Fox Film Corporation,NaN,NaN,Savannah 'Vannah' Jackson,Whitney Houston,Forest Whitaker,1995,5.090760
4,0.0,11862,Father of the Bride Part II,8.387519,1995-02-10,76578911.0,Father of the Bride Part II,5.7,173.0,Father of the Bride Collection,[Comedy],Sandollar Productions,Touchstone Pictures,NaN,George Banks,Steve Martin,Charles Shyer,1995,0.000000


In [110]:
df_movies.to_csv('movies.csv', index=False)